In [1]:
from datetime import datetime

import qnexus as qnx

from pytket import Circuit

from qnexus.client.models.nexus_dataclasses import QuantinuumConfig



In [2]:
my_project = qnx.project.get_only(name_like="VanyaTest")

qnx.context.set_active_project(my_project)

my_job_name_prefix = datetime.now()

## Compile Jobs

In [3]:
c = Circuit(2,2).H(0).CX(0,1).measure_all()

my_circuit_ref = qnx.circuit.upload(
    circuit=c,
    name="A circuit 17 May",
)

In [4]:
compile_job_ref = qnx.compile(
    circuits=[my_circuit_ref],
    name=f"{my_job_name_prefix}_compile",
    optimisation_level=1,
    target=QuantinuumConfig(device_name="H1-1SC")
)

In [5]:
qnx.job.wait_for(compile_job_ref)

JobStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='The job is completed.', error_detail=None, completed_time=datetime.datetime(2024, 5, 17, 13, 42, 15, 845125, tzinfo=datetime.timezone.utc), queued_time=None, submitted_time=datetime.datetime(2024, 5, 17, 13, 42, 1, 298374, tzinfo=datetime.timezone.utc), running_time=datetime.datetime(2024, 5, 17, 13, 42, 1, 445317, tzinfo=datetime.timezone.utc), cancelled_time=None, error_time=None, queue_position=None)

In [6]:
# Pretending we lost the compile_job_ref variable, we can retrieve it using filters

job_matches = qnx.job.get(name_like=f"{my_job_name_prefix}_compile")

my_job = job_matches.list()[0]

my_job.df()

In [6]:
# get results
compile_results = qnx.job.results(compile_job_ref)

compile_results.df()

,name,description,project,id
0,A circuit 17 May-compilation,,VanyaTest,5a59b5c4-57bd-4730-8740-cb8cb5dfce89


In [7]:
compile_results[0].df()

,name,description,project,id
0,A circuit 17 May-compilation,,VanyaTest,5a59b5c4-57bd-4730-8740-cb8cb5dfce89


In [8]:
circuit_ref = compile_results[0].get_output()

In [9]:
circuit_ref.download_circuit()

[PhasedX(3.5, 0.5) q[0]; PhasedX(0.5, 0.5) q[1]; ZZPhase(0.5) q[0], q[1]; Measure q[0] --> c[0]; PhasedX(0.5, 0) q[1]; Measure q[1] --> c[1]; ]

In [10]:
passes = compile_results[0].get_passes()


passes.df()

,pass name,input,output,id
0,DecomposeBoxes,A circuit 17 May,A circuit 17 May,64224151-ea78-4e52-87ae-6cfe5bd1c805
1,CustomPass,A circuit 17 May,A circuit 17 May,cb03d314-0878-4f28-ac55-b84ce6ff5566
2,SynthesiseTK,A circuit 17 May,A circuit 17 May-QuantinuumBackend-2,5fe2b59b-2d01-4935-9c6c-087e78d5b541
3,NormaliseTK2,A circuit 17 May-QuantinuumBackend-2,A circuit 17 May-QuantinuumBackend-2,f305b55e-83f5-4b98-85aa-f72148f97406
4,DecomposeTK2,A circuit 17 May-QuantinuumBackend-2,A circuit 17 May-QuantinuumBackend-3,544a1683-934c-4c7b-abdc-929f2f8e3115
5,RebaseCustomViaTK2,A circuit 17 May-QuantinuumBackend-3,A circuit 17 May-QuantinuumBackend-4,7633cb4b-ed72-4822-ad68-3e3c539d0eca
6,ZZPhaseToRz,A circuit 17 May-QuantinuumBackend-4,A circuit 17 May-QuantinuumBackend-4,77282ab9-429a-4811-9982-0cec124efcda
7,RemoveRedundancies,A circuit 17 May-QuantinuumBackend-4,A circuit 17 May-QuantinuumBackend-4,c3e61f95-ec15-4c51-a896-4ba254696ec2
8,SquashRzPhasedX,A circuit 17 May-QuantinuumBackend-4,A circuit 17 May-QuantinuumBackend-5,31ee90d9-dc74-4d0b-a614-6ef75b164f2d
9,RemoveRedundancies,A circuit 17 May-QuantinuumBackend-5,A circuit 17 May-QuantinuumBackend-final,140e668b-d840-45f8-a88e-9e4833cdc4f5


## Execute Jobs

In [11]:
compiled_circuits = [item.get_output() for item in compile_results]

In [12]:
execute_job_ref = qnx.execute(
    circuits=compiled_circuits,
    name=f"{my_job_name_prefix}1_execute",
    n_shots=[100]* len(compiled_circuits),
    target=QuantinuumConfig(device_name="H1-1SC"),
)

In [13]:
qnx.job.wait_for(execute_job_ref)

JobStatus(status=<StatusEnum.COMPLETED: 'Circuit has completed. Results are ready.'>, message='The job is completed.', error_detail=None, completed_time=datetime.datetime(2024, 5, 17, 13, 43, 10, 694090, tzinfo=datetime.timezone.utc), queued_time=None, submitted_time=datetime.datetime(2024, 5, 17, 13, 42, 45, 214305, tzinfo=datetime.timezone.utc), running_time=None, cancelled_time=None, error_time=None, queue_position=None)

In [14]:
execute_results = qnx.job.results(execute_job_ref)

execute_results.df()

,name,description,project,id
0,2024-05-17 14:42:00.1568051_execute,None,VanyaTest,5192ac12-653f-4c39-95f6-bb90013febb7


In [15]:
execute_results[0].download_backend_info()

BackendInfo(name='QuantinuumBackend snapshot', device_name='H1-1SC', version='0.33.0', architecture=<tket::FullyConnected, nodes=20>, gate_set={<OpType.Measure: 66>, <OpType.Reset: 68>, <OpType.Rz: 36>, <OpType.PhasedX: 71>, <OpType.Barrier: 8>, <OpType.ZZMax: 73>, <OpType.ClassicalExpBox: 107>, <OpType.ZZPhase: 76>, <OpType.TK2: 44>, <OpType.WASM: 14>, <OpType.SetBits: 15>, <OpType.CopyBits: 16>, <OpType.RangePredicate: 17>, <OpType.ExplicitPredicate: 18>, <OpType.ExplicitModifier: 19>, <OpType.MultiBit: 20>}, n_cl_reg=0, supports_fast_feedforward=True, supports_reset=True, supports_midcircuit_measurement=True, all_node_gate_errors=None, all_edge_gate_errors=None, all_readout_errors=None, averaged_node_gate_errors=None, averaged_edge_gate_errors=None, averaged_readout_errors=None, misc={'cl_reg_width': 32, 'max_n_shots': 10000, 'options': {}, 'system_type': 'syntax checker', 'wasm': True})

In [17]:
execute_results[0].download_result()

BackendResult(q_bits={},c_bits={c[0]: 0, c[1]: 1},counts=None,shots=[[0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]
 [0]],state=None,unitary=None,density_matrix=None)

In [ ]:
# Example of a job erroring

In [ ]:
execute_job_ref = qnx.execute(
    circuits=[my_circuit_ref],
    name=f"error_execute_{datetime.now()}",
    n_shots=[100],
    target=QuantinuumConfig(device_name="H1-1LE")
)

In [ ]:
qnx.job.wait_for(execute_job_ref)

In [ ]:
# Refactor backendconfig purely in the client for now


# Config
# simplify and improve
# talk to pytket about their thoughts going forward

# can we combine Aer configs into one?


# backendresult is going away 